# LangChain with Open Source LLM and Open Source Embeddings & LangSmith

In the following notebook we will dive into the world of Open Source models hosted on Hugging Face's [inference endpoints](https://ui.endpoints.huggingface.co/).

The notebook will be broken into the following parts:

- 🤝 Breakout Room #1:
  1. Set-up Hugging Face Infrence Endpoints
  2. Install required libraries
  3. Set Environment Variables
  4. Testing our Hugging Face Inference Endpoint
  5. Creating LangChain components powered by the endpoints
  6. Retrieving data from Arxiv
  7. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Set-up LangSmith
  2. Creating a LangSmith dataset
  3. Creating a custom evaluator
  4. Initializing our evaluator config
  5. Evaluating our RAG pipeline

# 🤝 Breakout Room #1

## Task 1: Set-up Hugging Face Infrence Endpoints

Please follow the instructions provided [here](https://github.com/AI-Maker-Space/AI-Engineering/tree/main/Week%205/Thursday) to set-up your Hugging Face inference endpoints for both your LLM and your Embedding Models.

## Task 2: Install required libraries

Now we've got to get our required libraries!

We'll start with our `langchain` and `huggingface` dependencies.



In [1]:
!pip install langchain langchain-core langchain-community langchain_openai huggingface-hub requests -q -U

Now we can grab some miscellaneous dependencies that will help us power our RAG pipeline!

In [2]:
!pip install arxiv pymupdf faiss-cpu -q -U

## Task 3: Set Environment Variables

We'll need to set our `HF_TOKEN` so that we can send requests to our protected API endpoint.

We'll also set-up our OpenAI API key, which we'll leverage later.



In [14]:
import os
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("HuggingFace Write Token: ")

In [15]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Task 4: Testing our Hugging Face Inference Endpoint

Let's submit a sample request to the Hugging Face Inference endpoint!

In [16]:
model_api_gateway = "https://iomz2pji3shyr91e.us-east-1.aws.endpoints.huggingface.cloud" # << YOUR ENDPOINT URL HERE

> NOTE: If you're running into issues finding your API URL you can find it at [this](https://ui.endpoints.huggingface.co/) link.

Here's an example:

![image](https://i.imgur.com/xSCV0xM.png)

In [17]:
import requests

max_new_tokens = 256
top_p = 0.9
temperature = 0.1

prompt = "Hello! How are you?"

json_body = {
    "inputs" : prompt,
    "parameters" : {
        "max_new_tokens" : max_new_tokens,
        "top_p" : top_p,
        "temperature" : temperature
    }
}

headers = {
  "Authorization": f"Bearer {os.environ['HF_TOKEN']}",
  "Content-Type": "application/json"
}

response = requests.post(model_api_gateway, json=json_body, headers=headers)
print(response.json())

[{'generated_text': " I'm doing well, thanks for asking! *smiles* It's great to see you here! *nods*\n\n\n"}]


## Task 5: Creating LangChain components powered by the endpoints

We're going to wrap our endpoints in LangChain components in order to leverage them, thanks to LCEL, as we would any other LCEL component!

### HuggingFaceEndpoint for LLM

We can use the `HuggingFaceEndpoint` found [here](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/huggingface_endpoint.py) to power our chain - let's look at how we would implement it.

In [20]:
from langchain.llms import HuggingFaceEndpoint

endpoint_url = (
    model_api_gateway
)

hf_llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
    task="text-generation"
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/sysadmin/.cache/huggingface/token
Login successful


Now we can use our endpoint like we would any other LLM!

In [21]:
hf_llm.invoke("Hello, how are you?")

"\n\nI'm just an AI, I don't have emotions, but I'm here to help you with any questions or problems you might have. Is there something specific you'd like to talk about or ask?"

### HuggingFaceInferenceAPIEmbeddings

Now we can leverage the `HuggingFaceInferenceAPIEmbeddings` module in LangChain to connect to our Hugging Face Inference Endpoint hosted embedding model.

In [22]:
embedding_api_gateway = "https://vykjqfwtoui2xd3w.us-east-1.aws.endpoints.huggingface.cloud" # << Embedding Endpoint API URL

In [23]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings_model = HuggingFaceInferenceAPIEmbeddings(api_key=os.environ["HF_TOKEN"], api_url=embedding_api_gateway)

In [24]:
embeddings_model.embed_query("Hello, welcome to HF Endpoint Embeddings")[:10]

[-0.019261347,
 0.015496692,
 -0.04624366,
 -0.021588588,
 -0.0099318465,
 0.00024534433,
 -0.033293247,
 -0.0010797717,
 0.027844762,
 0.011513001]

#### ❓ Question #1

What is the embedding dimension of your selected embeddings model?

1024

Source: https://www.percona.com/blog/create-an-ai-expert-with-open-source-tools-and-pgvector/

## Task 6: Retrieving data from Arxiv

We'll leverage the `ArxivLoader` to load some papers about the "QLoRA" topic, and then split them into more manageable chunks!

In [70]:
from langchain.document_loaders import ArxivLoader

docs = ArxivLoader(query="QLoRA", load_max_docs=5).load()
len(docs)

5

In [71]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
    length_function = len,
)

split_chunks = text_splitter.split_documents(docs)

In [72]:
len(split_chunks)

528

Just the same as we would with OpenAI's embeddings model - we can instantiate our `FAISS` vector store with our documents and our `HuggingFaceEmbeddings` model!

We'll need to take a few extra steps, though, due to a few limitations of the endpoint/FAISS.

We'll start by embeddings our documents in batches of `32`.

In [73]:
embeddings = []
for i in range(0, len(split_chunks) - 1, 32):
  embeddings.append(embeddings_model.embed_documents([document.page_content for document in split_chunks[i:i+32]]))

In [74]:
embeddings = [item for sub_list in embeddings for item in sub_list]

#### ❓ Question #2

Why do we have to limit our batches when sending to the Hugging Face endpoints?

We have to limit our batches due to following reasons:

Memory Constraints: Each request consumes memory on the server. If a batch is too large, it could exceed the server's memory limit and cause the request to fail.

Performance: Processing a large batch might take a long time and could lead to timeouts if the server is not able to respond quickly enough.

Rate Limiting: Hugging Face API might have rate limits to prevent abuse and ensure fair usage. If you send too many requests at once, you might hit these rate limits.

Stability: Smaller batches can lead to more stable and reliable results, as they reduce the risk of a single large request failing and causing all the data to be lost.

Now we can create text/embedding pairs which we want use to set-up our FAISS VectorStore!

In [75]:
from langchain.vectorstores import FAISS

text_embedding_pairs = list(zip([document.page_content for document in split_chunks], embeddings))

faiss_vectorstore = FAISS.from_embeddings(text_embedding_pairs, embeddings_model)

Next, we set up FAISS as a retriever.

In [76]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k" : 2})

Let's test it out!

In [77]:
faiss_retriever.get_relevant_documents("What optimizer does QLoRA use?")

[Document(page_content='Among these approaches, QLoRA (Dettmers\net al., 2023) stands out as a recent and highly\nefficient fine-tuning method that dramatically de-\ncreases memory usage. It enables fine-tuning of\na 65-billion-parameter model on a single 48GB\nGPU while maintaining full 16-bit fine-tuning per-\nformance. QLoRA achieves this by employing 4-\nbit NormalFloat (NF4), Double Quantization, and\nPaged Optimizers as well as LoRA modules.\nHowever, another significant challenge when uti-'),
 Document(page_content='the computational overhead traditionally associated with fine-tuning such models.\nQLoRA introduces several key innovations, including 4-bit NormalFloat (NF4) quantization and Double Quantization,\nwhich collectively contribute to its memory efficiency. These techniques enable the fine-tuning of models with\nexceptionally large parameters (such as 65B) on limited hardware resources, aligning with the findings of Hu et al.\n[2021].\n4')]

### Prompt Template

Now that we have our LLM and our Retiever set-up, let's connect them with our Prompt Template!

In [78]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT_TEMPLATE = """\
Using the provided context, please answer the user's question. If you don't know, say you don't know.

Context:
{context}

Question:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

#### ❓ Question #3

Does the ordering of the prompt matter?

Yes, the ordering of the prompt can matter when using language models like GPT-3 or GPT-4. These models generate text sequentially from left to right, so the order of the words in the prompt can influence the generated output.

For example, if you're asking a question, the model might generate different answers depending on how you phrase the question. Similarly, if you're providing a list of items as input, the order of the items might affect the model's output.

However, the impact of the prompt order can vary depending on the specific task and the model's training data. Some tasks might be more sensitive to the order of the prompt than others. It's often a good idea to experiment with different prompt orders to see what works best for your specific use case.

## Task 7: Creating a simple RAG pipeline with LangChain v0.1.0

All that's left to do is set up a RAG chain - and away we go!

In [79]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

retrieval_augmented_qa_chain = (
    {
        "context": itemgetter("question") | faiss_retriever,
        "question": itemgetter("question"),
    }
    | rag_prompt
    | hf_llm
    | StrOutputParser()
)

Let's test it out!

In [80]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLoRA?"})

'\nAnswer:\nQLoRA is a method for fine-tuning large language models (LLMs) that is made accessible to a broad range of users through the use of low-rank matrices and quantization techniques. According to the provided context, QLoRA is used in the hands of large corporations that do not release models or source code for auditing, and it has the potential to have a broadly positive impact on the finetuning of high-quality LLMs.'

# 🤝 Breakout Room #2

## Task 1: Set-up LangSmith

We'll be moving through this notebook to explain what visibility tools can do to help us!

Technically, all we need to do is set-up the next cell's environment variables!

In [81]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

Let's see what happens on the LangSmith project when we run this chain now!

In [82]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLoRA?"})

'\nAnswer:\nQLoRA is a method for fine-tuning large language models (LLMs) that is widely accessible and has a broadly positive impact. It is described in two documents provided in the context. According to the documents, QLoRA uses low-rank matrices in conjunction with quantization techniques to fine-tune LLMs, which significantly reduces the computational resources required for the fine-tuning process.'

We get *all of this information* for "free":

![image](https://i.imgur.com/8Wcpmcj.png)

> NOTE: We'll walk through this diagram in detail in class.

####🏗️ Activity #1:

Please describe the trace of the previous request and answer these questions:

1. How many tokens did the request use?

From the last step screenshot 299
From my LangSmith Run 326

2. How long did the `HuggingFaceEndpoint` take to complete?

From the last step screenshot 4.71s
From my LangSmith Run 3.11s

## Task 2: Creating a LangSmith dataset

Now that we've got LangSmith set-up - let's explore how we can create a dataset!

First, we'll create a list of questions!

In [83]:
from langsmith import Client

questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

Now we can create our dataset through the LangSmith `Client()`.

In [84]:
client = Client()
dataset_name = "QLoRA RAG Dataset"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    dataset_id=dataset.id
)

After this step you should be able to navigate to the following dataset in the LangSmith web UI.

![image](https://i.imgur.com/CdFYGTB.png)

## Task 3: Creating a custom evaluator

Now that we have a dataset - we can start thinking about evaluation.

We're going to make a `StringEvaluator` to measure "dopeness".

> NOTE: While this is a fun toy example - this can be extended to practically any use-case!

In [85]:
import re
from typing import Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.evaluation import StringEvaluator

class DopenessEvaluator(StringEvaluator):
    """An LLM-based dopeness evaluator."""

    def __init__(self):
        llm = ChatOpenAI(model="gpt-4", temperature=0)

        template = """On a scale from 0 to 100, how dope (cool, awesome, lit) is the following response to the input:
        --------
        INPUT: {input}
        --------
        OUTPUT: {prediction}
        --------
        Reason step by step about why the score is appropriate, then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = PromptTemplate.from_template(template) | llm

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "scored_dopeness"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain.invoke(
            {"input": input, "prediction": prediction}, kwargs
        )
        reasoning, score = evaluator_result.content.split("\n", maxsplit=1)
        score = re.search(r"\d+", score).group(0)
        if score is not None:
            score = float(score.strip()) / 100.0
        return {"score": score, "reasoning": reasoning.strip()}

## Task 4: Initializing our evaluator config

Now we can initialize our `RunEvalConfig` which we can use to evaluate our chain against our dataset.

> NOTE: Check out the [documentation](https://docs.smith.langchain.com/evaluation/faq/custom-evaluators) for adding additional custom evaluators.

In [86]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[DopenessEvaluator()],
    evaluators=[
        "criteria",
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria(
            {
                "AI": "Does the response feel AI generated?"
                "Response Y if they do, and N if they don't."
            }
        ),
    ],
)

## Task 5: Evaluating our RAG pipeline

All that's left to do now is evaluate our pipeline!

In [87]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=retrieval_augmented_qa_chain,
    evaluation=eval_config,
    verbose=True,
    project_name="HF RAG Pipeline - Evaluation - v3",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'HF RAG Pipeline - Evaluation - v3' at:
https://smith.langchain.com/o/2f142be8-b860-5498-aaf0-55a67fe7e836/datasets/821ecae2-5d40-4f8b-af56-3f0cd3f6b6d4/compare?selectedSessions=166250cb-b91c-4ce6-8762-228571fd2935

View all tests for Dataset QLoRA RAG Dataset at:
https://smith.langchain.com/o/2f142be8-b860-5498-aaf0-55a67fe7e836/datasets/821ecae2-5d40-4f8b-af56-3f0cd3f6b6d4
[------------------------------------------------->] 6/6

,feedback.helpfulness,feedback.harmfulness,feedback.AI,feedback.scored_dopeness,error,execution_time,run_id
count,6.000000,6.0,6.000000,6.000000,0,6.000000,6
unique,NaN,NaN,NaN,NaN,0,NaN,6
top,NaN,NaN,NaN,NaN,NaN,NaN,13b1f7bf-92e7-45db-bcee-092589c17bd3
freq,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,0.500000,0.0,0.333333,0.650000,NaN,3.763989,NaN
std,0.547723,0.0,0.516398,0.286356,NaN,1.938977,NaN
min,0.000000,0.0,0.000000,0.100000,NaN,2.500901,NaN
25%,0.000000,0.0,0.000000,0.625000,NaN,3.016871,NaN
50%,0.500000,0.0,0.000000,0.750000,NaN,3.098132,NaN
75%,1.000000,0.0,0.750000,0.837500,NaN,3.180236,NaN


{'project_name': 'HF RAG Pipeline - Evaluation - v3',
 'results': {'8926e38e-14b9-4202-86be-8fa75e39cae7': {'input': {'question': 'What optimizer is used in QLoRA?'},
   'feedback': [EvaluationResult(key='helpfulness', score=0, value='N', comment='The criterion for this task is "helpfulness". \n\nTo assess this, we need to consider whether the submission is helpful, insightful, and appropriate. \n\nThe submission provides an answer to the question, which is about the optimizer used in QLoRA. The answer given is "Paged Optimizers". \n\nHowever, without any context or further information, it\'s hard to determine if this answer is correct or not. \n\nIf the answer is correct, it could be considered helpful and appropriate. But if it\'s not correct, it wouldn\'t be helpful or appropriate. \n\nThe submission also doesn\'t provide any additional insight beyond the direct answer to the question. \n\nGiven these considerations, it\'s difficult to definitively say whether the submission meets t